 Articles and papers to look at:

https://aclanthology.org/W17-3502/  
https://aclanthology.org/D14-1074.pdf  
https://aclanthology.org/P17-4008.pdf  
http://computationalcreativity.net/iccc2012/wp-content/uploads/2012/05095-Colton.pdf  
https://era.ed.ac.uk/bitstream/handle/1842/314/IP040022.pdf?sequence=1  
https://aclanthology.org/2020.acl-main.223/  
https://arxiv.org/pdf/1610.09889.pdf  
https://www.researchgate.net/profile/Pablo-Gervas/publication/220831684_Poetry_generation_in_COLIBRI/links/61add74350e22929cd4e9ab6/Poetry-generation-in-COLIBRI.pdf 
http://nil.fdi.ucm.es/sites/default/files/GervasAISB2013CRC.pdf  

Here is an example of an RNN using Keras, only training on one document.

In [3]:
#!pip install pronouncing
import pronouncing
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Embedding
import matplotlib.pyplot as plt 
import nltk
from nltk.tokenize import word_tokenize
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 16.4 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=a957bdf76ffbcccedc1ee16db4076794e58d7953f2805431bb4a76d85f1b6e67
  Stored in directory: /root/.cache/pip/wheels/09/e8/c0/3606d42fdbf5f3871564eb6a353591a8f5deeed013fdb73921
Successfully built pronouncing


In [4]:
import pandas as pd

from google.colab import drive

mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

import os
drive_root = mount + "/My Drive/Stats"
  
create_drive_root = True
if create_drive_root:
  print("\nColab: making sure ", drive_root, " exists.")
  os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/Stats  exists.

Colab: Changing directory to  /content/gdrive/My Drive/Stats
/content/gdrive/My Drive/Stats


In [8]:
f = open("spring2021.txt", "r")
text = f.read()

In [27]:
nltk.download('punkt')
text = re.sub("\[.*\]", "", text)
text = re.sub("\s\s\s\s\s*", "", text)
text_tokens = word_tokenize(text)
print(text_tokens[:5])

['\ufeffBuddha', 'StatueOnce', 'there', 'was', 'a']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
print(text_tokens[99])
print(pronouncing.rhymes(text_tokens[99]))

on
['abidjan', 'abron', 'alcon', 'almazan', 'alon', 'amman', 'antoine', 'aton', 'azerbaijan', 'azerbaijan', 'balaban', 'baran', 'bataan', 'baton', 'bhutan', 'bon', 'bonn', 'bonne', 'bronn', 'buisson', 'buthan', "c'mon", 'caan', 'cadogan', 'caen', 'ceylon', 'ceylon', 'chandon', 'chiffon', 'chon', 'christiane', 'christianne', 'chuan', 'come-on', 'con', 'conn', 'conn.', 'cron', 'daane', 'dahn', 'dainippon', 'defcon', 'dejohn', 'denouement', 'dhaharan', 'dhahran', 'dohn', 'don', 'donn', 'dwan', 'eldon', 'elston', 'erbakan', 'evonne', 'fernand', 'forgone', 'frohn', 'gabon', 'gaitan', 'galvan', 'gibran', 'gohn', 'goings-on', 'golan', 'grandmaison', 'guei-yuan', 'guinan', 'haan', 'hahn', 'han', 'hassan', 'hohn', 'hon', 'honn', 'hron', 'hron', 'huan', 'hwan', 'ilan', 'iran', 'irelan', 'jaan', 'javan', 'johan', 'john', 'jon', 'jonn', 'juan', 'juan', 'juran', 'jwan', 'kahan', 'kahn', 'kazakhstan', 'khan', 'kohn', 'kon', 'koran', 'kron', 'kuan', 'kwan', 'kwon', 'lacan', 'lat-lon', 'lian', 'livan'

In [29]:
vocabulary = np.array(sorted(set(text)))
tokenizer = {char:i for i,char in enumerate(vocabulary)}

In [30]:
vector = np.array([tokenizer[char] for char in text])
print('\nSample Text \n')
print('-'*70)
print(text[:100])
print('-'*70)
print('\n\nCorresponding Integer Vector \n')
print('-'*70)
print(vector[:100])
print('-'*70)


Sample Text 

----------------------------------------------------------------------
﻿Buddha StatueOnce there was a person who said to me in a mocking yet not unserious tone: "There is 
----------------------------------------------------------------------


Corresponding Integer Vector 

----------------------------------------------------------------------
[97 25 71 54 54 58 51  1 42 70 51 70 71 55 38 64 53 55  1 70 58 55 68 55
  1 73 51 69  1 51  1 66 55 68 69 65 64  1 73 58 65  1 69 51 59 54  1 70
 65  1 63 55  1 59 64  1 51  1 63 65 53 61 59 64 57  1 75 55 70  1 64 65
 70  1 71 64 69 55 68 59 65 71 69  1 70 65 64 55 21  1  3 43 58 55 68 55
  1 59 69  1]
----------------------------------------------------------------------


In [31]:
vector = tf.data.Dataset.from_tensor_slices(vector)
sequences = vector.batch(100, drop_remainder=True)

def prepare_dataset(seq):
     input_vector = seq[:-1]
     target_vector = seq[1:]
     return input_vector, target_vector
dataset = sequences.map(prepare_dataset)

In [32]:
AUTOTUNE = tf.data.AUTOTUNE
 # buffer size 10000
 # batch size 64
data = dataset.batch(64, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)
 # steps per epoch is number of batches available
STEPS_PER_EPOCH = len(sequences)//64
for inp, tar in data.take(1):
    print(inp.numpy().shape)
    print(tar.numpy().shape)

(64, 99)
(64, 99)


In [33]:
model = keras.Sequential([
     # Embed len(vocabulary) into 64 dimensions
     Embedding(len(vocabulary), 64, batch_input_shape=[64,None]),
     # LSTM RNN layers
     LSTM(512, return_sequences=True, stateful=True),
     LSTM(512, return_sequences=True, stateful=True),
     # Classification head
     Dense(len(vocabulary))
 ])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 64)            6272      
                                                                 
 lstm_2 (LSTM)               (64, None, 512)           1181696   
                                                                 
 lstm_3 (LSTM)               (64, None, 512)           2099200   
                                                                 
 dense_1 (Dense)             (64, None, 98)            50274     
                                                                 
Total params: 3,337,442
Trainable params: 3,337,442
Non-trainable params: 0
_________________________________________________________________


In [34]:
for example_inp, example_tar in data.take(1):
     example_pred = model(example_inp)
     print(example_tar.numpy().shape)
     print(example_pred.shape)

(64, 99)
(64, 99, 98)


In [35]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [42]:
#gpu necessary for this many epochs
model.compile(optimizer='adam', 
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

history = model.fit(data, 
                     epochs=100, 
                     steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/100
13/13 [==============================] - 3s 60ms/step - loss: 0.0660
Epoch 2/100
13/13 [==============================] - 1s 53ms/step - loss: 0.0104
Epoch 3/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0063
Epoch 4/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0040
Epoch 5/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0027
Epoch 6/100
13/13 [==============================] - 1s 52ms/step - loss: 0.0018
Epoch 7/100
13/13 [==============================] - 1s 53ms/step - loss: 0.0014
Epoch 8/100
13/13 [==============================] - 1s 53ms/step - loss: 0.0012
Epoch 9/100
13/13 [==============================] - 1s 53ms/step - loss: 0.0012
Epoch 10/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0011
Epoch 11/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0012
Epoch 12/100
13/13 [==============================] - 1s 51ms/step - loss: 0.0013
Epoch 13/100
13/13 [=====

In [43]:
sample = 'Chickadees '
 # vectorize the string
sample_vector = [tokenizer[s] for s in sample]
predicted = sample_vector
 # convert into tensor of required dimensions
sample_tensor = tf.expand_dims(sample_vector, 0) 
 # broadcast to first dimension to 64 
sample_tensor = tf.repeat(sample_tensor, 64, axis=0)

 # predict next 1000 characters
 # temperature is a sensitive variable to adjust prediction
temperature = 0.6

for i in range(1000):
     pred = model(sample_tensor)
     # reduce unnecessary dimensions
     pred = pred[0].numpy()/temperature
     pred = tf.random.categorical(pred, num_samples=1)[-1,0].numpy()
     predicted.append(pred)
     sample_tensor = predicted[-99:]
     sample_tensor = tf.expand_dims([pred],0)
     # broadcast to first dimension to 64 
     sample_tensor = tf.repeat(sample_tensor, 64, axis=0)

 # convert the integers back to characters
pred_char = [vocabulary[i] for i in predicted]
generated = ''.join(pred_char) 
print(generated)

Chickadees reading away the little girl. The is airechality be our handso mow me endering a correr bart. I saud yer sich.
She conces from the same soil as my mother, a Portuguese term, one that holds many definitions. What eta of the ancohol and the noise, she can’t stop thinking of that sparse room, of mold hald but As the Popticiesly lieet the uarth of the street. Where half the older people are still illiterate and then’there blost misick of the same greef. The build was gostal wifh where her carainet marrished before the congertion. This is because, in walking such a path, to peer ahead of oneself is to faccied the other against the smot is the die,
Is tof the hay they sped down the street. Where half the older people are still illiterate and then’there becomes, but the forest asking flow rowher that we are had and so againg
to the street lamps turned on. As I would sink deeper into the fantasy, my mother’s dreams. She beaves all her all
this world jo there in towards to prace thei